In [1]:
import tensorflow as tf
print("tensorflow version " + str(tf.__version__)) # tested on 2.4.1
import tensorflow_addons as tfa
print("tensorflow-addons version " + str(tfa.__version__))
from tensorboard import notebook
%load_ext tensorboard

from datetime import datetime

from dense_image_warp import *
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
w=128
h=128

batch_size=8*4
imgs=tf.random.poisson((batch_size,w,h,3),lam=0.5)

In [ ]:
scale = 5

x = tf.linspace(0.0,1.0/scale*(w-1),w)
y = tf.linspace(0.0,1.0/scale*h-1,h)
grid_x, grid_y= tf.meshgrid(x,y)
stacked_grid = tf.cast(tf.stack([grid_y, grid_x], axis=2), float)
batched_grid = tf.broadcast_to(stacked_grid,(batch_size,)+stacked_grid.shape)
query_points_on_grid = batched_grid+64
print(tf.shape(batched_grid))
query_points_flattened = tf.reshape(
    query_points_on_grid, [batch_size, h * w, 2]
)

In [ ]:
#tf.profiler.experimental.start('logtest/catmull')
res=interpolate_catmull_rom(imgs,query_points_flattened)
#tf.profiler.experimental.stop()

In [ ]:
#tf.profiler.experimental.start('logtest/bilin')
res2=interpolate_bilinear(imgs,query_points_flattened)
#tf.profiler.experimental.stop()

In [ ]:
res=tf.reshape(res, [batch_size, h, w, 3])
res2=tf.reshape(res2, [batch_size, h, w, 3])
plt.imshow(imgs[0])
plt.figure()
plt.imshow(res[0])
plt.figure()
plt.imshow(res2[0])

In [ ]:
test=np.ones((2, 2, 2250))*0.25
print(test[0,0,0:15])
print("_____")
print(test[0,1,0:15])
print("_____")
print("_____")
test = test-1
print(test[0,0,0:15])
print("_____")
print(test[0,1,0:15])
print("_____")
print("_____")

In [ ]:
def gather(y_coords, x_coords, name):
    with tf.name_scope("gather-" + name):
        linear_coordinates = batch_offsets + y_coords * width + x_coords
        gathered_values = tf.gather(flattened_grid, linear_coordinates)
        return tf.reshape(gathered_values, [batch_size, num_queries, channels])

In [ ]:
np.shape(test[0])

In [ ]:

stacktest = np.ones((5, 225, 3))
stacked=np.stack([stacktest,stacktest],  axis=-1)

print(np.shape(stacked))
reduced=np.sum(stacked,axis=-1)
print(np.shape(reduced))

In [ ]:
aye=tf.zeros((5,255,3,2))
aye[:,:,:,0]=stacktest
aye[:,:,:,1]=stacktest